In [40]:
import pandas as pd
import pyarrow.parquet as pq

In [41]:
# Cargar las tablas desde los archivos parquet
steam_games = pq.read_table("steam_games.parquet").to_pandas()
users_items = pq.read_table("users_items.parquet").to_pandas()

In [42]:
def PlayTimeGenre(genre):
    # Convertir 'item_id' a tipo numérico si es necesario
    users_items['item_id'] = pd.to_numeric(users_items['item_id'], errors='coerce')

    # Unir las tablas utilizando la columna "id" en 'steam_games' y "item_id" en 'users_items'
    merged_data = pd.merge(users_items, steam_games, left_on="item_id", right_on="id", how="inner")

    # Filtrar los juegos que correspondan al género proporcionado
    genre_data = merged_data.dropna(subset=['genres'])
    genre_data = genre_data[genre_data['genres'].apply(lambda x: genre in x)]

    # Convertir la columna "release_date" a tipo datetime
    try:
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)
    except ValueError:
        # Si la inferencia de formato falla, intentar especificar el formato manualmente
        genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='coerce', format='%b %Y')

    # Extraer el año de la columna "release_date"
    genre_data['year'] = genre_data['release_date'].dt.year

    # Agrupar por año y calcular la suma de las horas jugadas para cada año
    result = genre_data.groupby('year')['playtime_forever'].sum().reset_index()

    # Encontrar el año con la mayor cantidad de horas jugadas
    max_year = result.loc[result['playtime_forever'].idxmax()]['year']

    return max_year

In [43]:
# Ejemplo de uso
genre = "Casual"  # Reemplaza con el género deseado
result = PlayTimeGenre(genre)
print(f"El año con más horas jugadas para el género {genre} es: {result}")

/var/folders/7l/jr096cm117gfqxv28vmgt9b80000gp/T/ipykernel_46705/2675117312.py:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  genre_data['release_date'] = pd.to_datetime(genre_data['release_date'], errors='raise', infer_datetime_format=True)


El año con más horas jugadas para el género Casual es: 2014.0
